# Simple Ten Node Cluster: Jobs Traces Replay

Simple Ten Node Cluster is homoginious cluster containing only ten nodes. The scheduler it simple First Come First Served with a backfiller.

Here we will examing the effect of backfiller.


In [1]:
# Load Used Packages
using Random
using Agents
using HPCMod
using Printf
using DataFrames
using DataStructures

`job_trace` specified jobs which will be submitted. We use `DataFrame` to specify parameters:
* `submit_time` - submit time in hours
* `user_id` - user id
* `nodes` - requested number of nodes
* `walltime`- requested walltime

In [2]:
job_trace = DataFrame([
    2 1 2 2;
    3 2 2 2;
    4 2 3 3;
    5 2 2 2;
    5 1 2 2;
    6 1 1 3;
], [
    "submit_time", "user_id", "nodes", "walltime"
])

Row,submit_time,user_id,nodes,walltime
,Int64,Int64,Int64,Int64
1,2,1,2,2
2,3,2,2,2
3,4,2,3,3
4,5,2,2,2
5,5,1,2,2
6,6,1,1,3


## Run without Backfill Scheduler

In [3]:
sim = jobs_replay_on_resource(
    job_trace; 
    nodes=4, 
    scheduler_backfill=false, 
    workload_done_check_freq=1, 
    rng=Random.Xoshiro(123))
println(sim.resource.stats.node_occupancy_by_job)

14×5 DataFrame
 Row │ t      N0001  N0002  N0003  N0004 
     │ Int64  Int64  Int64  Int64  Int64 
─────┼───────────────────────────────────
   1 │     0      0      0      0      0
   2 │     1      0      0      0      0
   3 │     2      1      1      0      0
   4 │     3      1      1      2      2
   5 │     4      0      0      2      2
   6 │     5      3      3      3      0
   7 │     6      3      3      3      0
   8 │     7      3      3      3      0
   9 │     8      5      5      4      4
  10 │     9      5      5      4      4
  11 │    10      6      0      0      0
  12 │    11      6      0      0      0
  13 │    12      6      0      0      0
  14 │    13      0      0      0      0


The tables above show nodes occupience by job at a given time.
0-means no jobs running on that node, otherwise show job id.

## Lets Try Same But with Backfiller

In [4]:
sim = jobs_replay_on_resource(
    job_trace; 
    nodes=4, 
    scheduler_backfill=true, 
    workload_done_check_freq=1, 
    rng=Random.Xoshiro(123))

println(sim.resource.stats.node_occupancy_by_job)

12×5 DataFrame
 Row │ t      N0001  N0002  N0003  N0004 
     │ Int64  Int64  Int64  Int64  Int64 
─────┼───────────────────────────────────
   1 │     0      0      0      0      0
   2 │     1      0      0      0      0
   3 │     2      1      1      0      0
   4 │     3      1      1      2      2
   5 │     4      0      0      2      2
   6 │     5      3      3      3      0
   7 │     6      3      3      3      6
   8 │     7      3      3      3      6
   9 │     8      5      5      0      6
  10 │     9      5      5      4      4
  11 │    10      0      0      4      4
  12 │    11      0      0      0      0


Job 6 was scheduler earlier through backfill